In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('X_train.csv')

## Признаки

Закодировали one-hot категориальные признаки и снизили размерность с помощью метода главных компонент

In [4]:
cols_to_transform = ['sku', 'categoryLevel1Id', 'categoryLevel2Id', 'brandId', 'userName']

In [5]:
full2 = pd.get_dummies(columns = cols_to_transform, data=df[cols_to_transform])

In [6]:
from sklearn.decomposition import PCA

In [7]:
pca_cat = PCA(n_components=1000)
pca_cat = pca_cat.fit_transform(full2)

Тексты комментариев лемматизируем с помощью mystem3, посчитаем матрицу tf-idf и также снизим ее размерность с помощью метода главных компонент

In [221]:
output_file = open('comments.txt', 'w')
for text in df.comment.values:
    output_file.write(text + '\n')
output_file.close()

In [16]:
! ./mystem -cl comments.txt norm_comments2.txt

In [18]:
import re

In [19]:
f = open('norm_comments.txt', 'r')
prog = re.compile('{[А-Яа-я\|]+}')
norm_texts = []
for line in f.readlines():
    norm_texts.append([l[1:-1].split("|") for l in prog.findall(line.lower())])
f.close()

In [20]:
norm_texts2 = [' '.join([word[0] for word in text]) for text in norm_texts]

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tiv = TfidfVectorizer()
nwd = tiv.fit_transform(norm_texts2)

In [22]:
pca = PCA(n_components=1000)
pca_data = pca.fit_transform(nwd.todense())

Добавим еще несколько признаков:

1. средняя тональность слов в комментарии (учитываются только те слова, у которых тональность в словаре ... не ноль)

2. количество слов

3. Веселые и грустные смайлики

4. Количство предложений

5. Количество восклицательных знаков

In [25]:
sent = pd.read_csv('words.csv', sep=';', header=None, names =['word', 'sent', '1', '2', '3'])

In [26]:
Dict = sent.groupby('word')['sent'].mean().to_dict()

In [27]:
tone = np.zeros((len(norm_texts), 6))

In [28]:
for i, text in enumerate(norm_texts):
    count = 0
    for j, word in enumerate(text):
        s = Dict.get(word[0], 0)
        if s != 0:
            count += 1
            if j >= 1:
                if text[j-1][0] == 'не':
                    s = - s
            tone[i, 0] += s
    if count:
        tone[i, 0] = tone[i, 0]/count

In [29]:
for i, text in enumerate(norm_texts):
    tone[i, 1] = len(text)

In [30]:
for i, text in enumerate(df.comment):
    left = 0
    right = 0
    for j, word in enumerate(text):
        if word == ')':
            right += 1
        if word == '(':
            left += 1
    if (right - left)%2 == 1:
        if right - left > 0:
            tone[i, 2] = 1
        else:
            tone[i, 3] = 1

In [31]:
from nltk.tokenize import sent_tokenize

In [32]:
for i, text in enumerate(df.comment):
    sents = sent_tokenize(text)
    tone[i, 4] = len(sents)

In [33]:
for i, text in enumerate(df.comment):
    for e in text:
        if e == '!':
            tone[i, 5] += 1

## Целевая переменная

Так как практически все оценки, целые задачу можно сформулировать как многоклассовую классификацию.

In [34]:
y = list(map(int, np.round(df.reting)))

## Обучение классификатора

Разобьем выборку на обучающую и тестовую и обучим классификаторы с параметрами, подобранными на кросс-валидации. Обучим логистическую регрессию one vs rest с разрежеванием (l1 регуляризатор) и далее обучим логистическую регрессию с l2 регуляризатором и оценим качество на кросс-валидации и отложенной выборке.

In [35]:
train = np.hstack((pca_cat, pca_data, tone))

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.1, random_state=42)

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
model1 = LogisticRegression(multi_class="ovr", penalty='l1', C=1.1)
model1.fit(X_train, y_train)

LogisticRegression(C=1.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
ind = (model1.coef_[0, :] == 0) & (model1.coef_[1, :] == 0) & (model1.coef_[2, :] == 0) & (model1.coef_[3, :] == 0) & (model1.coef_[4, :] == 0)

In [41]:
model2 = LogisticRegression(multi_class="ovr", penalty='l2', C=1.3) #0.89
model2.fit(X_train[:, ~ind], y_train)

LogisticRegression(C=1.3, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Оценим качество

Так как выборка сильно смещена в сторону пяти (примерно половина отзывов имеют рейтинг пять), будем использовать микро усредненную f-меру для сравнения качества работы алгоритмов.

In [42]:
from sklearn.metrics import f1_score

In [43]:
y_train_pr1 = model1.predict(X_train)
y_test_pr1 = model1.predict(X_test)

In [44]:
y_train_pr2 = model2.predict(X_train[:, ~ind])
y_test_pr2 = model2.predict(X_test[:, ~ind])

In [45]:
f1_score(y_true=y_test, y_pred=y_test_pr2, average='micro')

0.66132135984605511

In [46]:
f1_score(y_true=y_train, y_pred=y_train_pr2, average='micro')

0.74315654405474763

In [51]:
from sklearn.model_selection import cross_val_score

In [52]:
model = LogisticRegression(multi_class="ovr", penalty='l2', C=1.3)
scores = cross_val_score(model, X_train[:, ~ind], y_train, cv=5, scoring='f1_micro')
scores.mean()

0.66125045521371228